In [ ]:
import pandas as pd
import numpy as np
import pywhatkit as wpp
from datetime import datetime 


#SetOptions para ver los dataframes
#pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth',False)

In [ ]:
path = 'Data/'

mes = datetime.now().month

df_matries = pd.read_excel(path + 'Román - M{:d} Seguimiento.xlsx'.format(mes), sheet_name='Asignación')
database = pd.read_excel(path + "BASE CNCI.xlsx", sheet_name = 'Activos')
#inactivos = pd.read_excel(path + 'Alertas tempranas {:2} de Abril 2024.xlsx'.format(dia), sheet_name='Inactivo')

In [ ]:
#Renombrar el nombre de la columna NOMBRE
database.rename(columns = {'NOMBRE                                                                                                                                      ' : 'Nombre'}, inplace= True)

#Solo las columnas necesarias
database = database[['Nombre', 'MATRICULA', 'CE CANALIZADO', 'CARRERA', 'M4', 'Correo', 'Teléfono', 'Teléfono2', 'Teléfono3', 'CURP', 'Agente ', 'Asesor Académico', 'Asesor Financiero ', 'Asesor Escolar']]

#Filtro Cadereyta
database = database[database['CE CANALIZADO'] == 'CE CADEREYTA']
database.columns

#### Inactivos DB

In [ ]:
# Filtro Cadereyta
df_matries = df_matries[df_matries['CE'] == 'CE CADEREYTA']

#Se pone el uppercase a las matriculas
df_matries['MATRICULA'] = df_matries['MATRICULA'].str.upper()
df_matries.shape

##### Para Alumnos de (tal materia)

In [ ]:
path2 = 'infodata/'
link = 'https://chat.whatsapp.com/Env3dGXvLIl7HYXWFUtICE'
imagen = 'img/FunMat.png'
df_matries = df_matries[df_matries['MATERIA'] == 'Fundamentos de matemáticas']

# Inicializar la infodata
try:
    matric_error = pd.read_csv(path2 + 'matriculas_error.csv')
    matric_yacontactadas = pd.read_csv(path2+ 'matriculas_yacontactadas.csv') 
    
    matric_error = matric_error.dropna().values.flatten().tolist()
    matric_yacontactadas = matric_yacontactadas.dropna().values.flatten().tolist() 
except FileNotFoundError:
    matric_error = pd.DataFrame()
    matric_yacontactadas = pd.DataFrame()

In [ ]:
# Materia del Grupo Colab
df_matries = df_matries[df_matries['MATERIA'] == 'Fundamentos de matemáticas']

# Merge nuestros df, con el fin de no llamar a dos df en el for.
df_matries = pd.merge(df_matries, database[['MATRICULA','Teléfono', 'Teléfono2', 'M4']], on = 'MATRICULA')
df_matries.columns

#### Ciclo de mensajes

In [ ]:
for index, row in df_matries.iterrows():
    alumno = row['NOMBRE']
    matricula = str(row['MATRICULA'].upper())
    materia = row['MATERIA']
    telefono = row['Teléfono']
    
    if not np.isnan(telefono):
        telefono = '+52 ' + str(telefono)
    if matricula in matric_error:
        if not np.isnan(row['Teléfono2']):
            telefono2 = '+52 ' + str(row['Teléfono2'])
            telefono = telefono2
            print('Matricula con error: {}, Alumno: {}'.format(matricula, alumno))       
    


    if matricula in matric_yacontactadas:
        print('Matricula antes contactada: {}, Alumno: {} '.format(matricula, alumno))    
    else:
        mensaje = 'Hola, que tal ' + alumno + ' ' + matricula + ', soy _Román Rodríguez_ tu Tutor Presencial del CNCI, me comunico contigo para invitarte al grupo colaborativo de la materia: *'+ materia + '* con la finalidad de que podamos entablar conversación por ahí sobre los temas relacionados y además tener apoyo entre los compañeros. \n\nSi gustas a continuación te dejo el enlace: ' + link
        print('Enviado a: ', matricula, alumno)
        print('\n')
        #wpp.sendwhats_image(telefono, imagen, mensaje, 25, True)
        #print(telefono, matricula, alumno)